# 식품 라벨 분석 파이프라인: OCR 텍스트 추출 및 알레르겐 분석

이 노트북은 이미지 형태의 식품 라벨을 분석하는 두 가지 주요 파이프라인을 구현합니다.

1.  **단순 텍스트 추출 (OCR → .txt)**: 이미지에서 모든 텍스트를 추출하여 `.txt` 파일로 저장합니다.
2.  **알레르겐 분석 (OCR → LLM → .json)**: 이미지의 텍스트를 기반으로 한국 식약처(MFDS) 기준 알레르겐 정보를 분석하고, 구조화된 `.json` 파일로 결과를 저장합니다.

## 1. 사전 준비 (Prerequisites)

아래 셀을 실행하여 파이프라인에 필요한 모든 라이브러리를 설치합니다.

In [1]:
!pip install google-cloud-vision
!pip install transformers accelerate torch tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 370.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 398.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 275.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 187.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 311.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 359.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 282.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


### Google Cloud Platform (GCP) 인증 설정

GCP Vision API를 사용하려면 서비스 계정 키 파일(`.json`)이 필요합니다.
아래 코드에서 `YOUR_SERVICE_ACCOUNT_KEY.json` 부분을 실제 키 파일 경로로 수정하세요.

In [2]:
import os

# 🚨 중요: 이 부분을 실제 GCP 서비스 계정 키 파일 경로로 변경하세요.
key_path = "ocr-project-470906-7ffeebabeb09.json"

if os.path.exists(key_path):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path
    print(f"✅ GCP 인증 설정 완료: {key_path}")
else:
    print(f"⚠️ 경고: GCP 인증 키 파일을 찾을 수 없습니다. 경로를 확인하세요: {key_path}")

✅ GCP 인증 설정 완료: ocr-project-470906-7ffeebabeb09.json


## 2. 라이브러리 임포트 및 전역 설정

파이프라인 전체에서 사용할 라이브러리와 주요 설정값을 정의합니다.

In [3]:
# 기본 라이브러리
import io
import json
import re
import textwrap
import argparse
from typing import List, Dict, Set

# Google Cloud Vision API
from google.cloud import vision

# Hugging Face Transformers (LLM)
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- 전역 설정 ---

# 사용할 LLM 모델 이름 (VRAM이 부족하면 더 작은 모델로 변경 가능)
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# 한국 식약처(MFDS) 고시 알레르겐 표준 명칭
MFDS_CANON = [
    "계란(난류)","우유","메밀","땅콩","대두","밀","고등어","게","새우",
    "돼지고기","복숭아","토마토","아황산류","호두","닭고기","쇠고기",
    "오징어","조개류","잣"
]

# LLM 모델과 토크나이저를 저장할 전역 변수 (메모리에 한 번만 로드하기 위함)
_tokenizer = None
_model = None

## 3. 핵심 기능: 이미지 텍스트 추출 (GCP Vision OCR)

이 함수는 두 파이프라인 모두에서 사용하는 가장 기본적인 기능입니다. 이미지 파일 경로를 받아 GCP Vision API를 통해 텍스트를 추출하고 문자열로 반환합니다.

In [4]:
def detect_text_by_gcp(image_path: str) -> str:
    """
    이미지 파일에서 텍스트를 감지하여 하나의 문자열로 반환합니다.

    Args:
        image_path (str): 분석할 이미지 파일의 경로.

    Returns:
        str: 추출된 전체 텍스트. 텍스트가 없으면 빈 문자열을 반환.
        
    Raises:
        FileNotFoundError: 이미지 파일 경로가 잘못된 경우 발생.
        RuntimeError: GCP API 호출 시 에러가 발생한 경우.
    """
    # 1. 파일 존재 여부 확인
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"이미지 파일을 찾을 수 없습니다: {image_path}")

    print(f"'{os.path.basename(image_path)}' 파일에서 텍스트를 추출합니다...")
    
    try:
        # 2. GCP Vision 클라이언트 초기화
        client = vision.ImageAnnotatorClient()
        
        # 3. 이미지 파일을 바이너리(binary) 모드로 읽기
        with io.open(image_path, "rb") as image_file:
            content = image_file.read()

        # 4. GCP API가 인식할 수 있는 이미지 형식으로 변환
        image = vision.Image(content=content)
        
        # 5. text_detection API 호출
        response = client.text_detection(image=image)

        # API 응답에 에러 메시지가 있는지 확인
        if response.error.message:
            raise RuntimeError(
                f"GCP OCR 오류: {response.error.message}\n"
                "자세한 내용: https://cloud.google.com/apis/design/errors"
            )

        # 6. 결과 파싱
        texts = response.text_annotations
        if not texts:
            print("이미지에서 텍스트를 찾지 못했습니다.")
            return ""
        
        print("텍스트 추출 완료.")
        # response.text_annotations의 첫 번째 요소(index 0)에 전체 텍스트가 포함됨
        return texts[0].description

    except Exception as e:
        print(f"API 호출 중 오류가 발생했습니다: {e}")
        print("Google Cloud 인증(GOOGLE_APPLICATION_CREDENTIALS)이 올바르게 설정되었는지 확인하세요.")
        return ""

## 4. 파이프라인 1: 알레르겐 분석 (OCR → LLM → JSON)

텍스트 추출 후, LLM을 통해 알레르겐 정보를 분석하고 JSON으로 결과를 정제하는 전체 과정입니다.

### 4-1. LLM 로딩 및 프롬프트 정의

LLM을 메모리에 로드하고, LLM에게 역할을 부여하는 시스템 프롬프트를 설정합니다.

In [5]:
def load_llm():
    """Hugging Face LLM과 토크나이저를 로드합니다. 이미 로드된 경우 건너뜁니다."""
    global _tokenizer, _model
    if _model is None:
        print(f"'{MODEL_NAME}' 모델을 로딩합니다. 잠시 기다려주세요...")
        _tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
        _model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            device_map="auto",  # 사용 가능한 GPU/CPU에 자동으로 모델을 할당
            trust_remote_code=True,
            torch_dtype="auto", # 사용 가능한 하드웨어에 맞춰 데이터 타입 자동 설정
        )
        print("모델 로딩 완료.")

# LLM에게 역할을 부여하고, 출력 형식을 지정하는 시스템 프롬프트
SYSTEM_PROMPT = textwrap.dedent("""
너는 '한국 식품 라벨 분석가'야. 입력은 GCP OCR로 추출한 라벨 텍스트다.
목표: 한국 식약처(MFDS) 알레르기 표시 기준 안에서 포함 여부를 판별해 JSON만 출력한다.

규칙:
- "알레르기 유발물질", "함유" 등 명시 라인을 최우선으로 사용한다.
- 명시 라인이 없으면 원재료명에서 동의어를 근거로 추론한다.
- 결과는 반드시 아래 스키마의 JSON 형식으로만 출력하며, 다른 설명은 절대 추가하지 않는다.
- evidence는 원문에서 찾은 '최소한의 문자열'만 사용한다. 예: "우유", "치즈분말".
- "… 대두, 밀, 우유 … 함유" 문장은 각 알레르겐으로 분할하여 해당 단어만 evidence로 쓴다.

JSON 스키마:
{
  "allergens_found": [
    {"canonical": "<MFDS 표준명>", "evidence": ["<근거1>", "<근거2>"]}
  ],
  "uncertain_mentions": ["<애매하거나 교차오염 가능성 표기>"],
  "notes": "<판별 근거 요약(한 줄)>"
}
""").strip()

def build_user_prompt(ocr_text: str) -> str:
    """OCR 텍스트를 받아 LLM에게 전달할 최종 사용자 프롬프트를 생성합니다."""
    return textwrap.dedent(f"""
    다음은 GCP Vision OCR로 추출한 식품 라벨 텍스트입니다.
    시스템 프롬프트의 규칙을 적용해 JSON만 출력해 주세요.

    [OCR_TEXT_START]
    {ocr_text.strip()}
    [OCR_TEXT_END]
    """).strip()

### 4-2. LLM 추론 및 결과 파싱

LLM을 실행하고, 모델이 생성한 텍스트 응답에서 깨끗한 JSON 데이터만 안전하게 추출합니다.

In [6]:
def llm_infer_to_json(ocr_text: str) -> Dict:
    """OCR 텍스트로 LLM 추론을 실행하고, 결과 텍스트를 JSON(Dict)으로 파싱합니다."""
    load_llm() # 모델이 로드되었는지 확인
    
    # 시스템 프롬프트와 사용자 프롬프트를 LLM이 이해하는 대화 형식으로 조합
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(ocr_text)}
    ]
    prompt = _tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # 텍스트를 모델이 처리할 수 있는 텐서(Tensor)로 변환
    inputs = _tokenizer(prompt, return_tensors="pt").to(_model.device)
    
    # LLM 텍스트 생성 실행
    outputs = _model.generate(
        **inputs,
        max_new_tokens=800,   # 최대 생성 토큰 수
        do_sample=False,      # 항상 가장 확률 높은 단어만 선택 (결과의 일관성 확보)
        temperature=0.0,      # 창의성 0 (결과의 일관성 확보)
        eos_token_id=_tokenizer.eos_token_id,
    )
    
    # 생성된 텐서를 다시 사람이 읽을 수 있는 텍스트로 변환
    full_response = _tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 모델 응답 부분만 잘라내기
    assistant_response = full_response.split("assistant\n")[-1].strip()
    
    # 텍스트에서 JSON 부분만 안전하게 파싱
    return safe_json_parse(assistant_response)


def safe_json_parse(model_text: str) -> Dict:
    """모델이 출력한 텍스트에서 JSON 블록만 추출해 안전하게 파싱합니다."""
    # 모델이 ```json ... ``` 코드 블록으로 응답하는 경우를 먼저 처리
    match = re.search(r"```json\s*(\{.*?\})\s*```", model_text, re.DOTALL)
    if match:
        chunk = match.group(1)
    else:
        # 코드 블록이 없다면, 가장 바깥쪽의 중괄호({})를 기준으로 JSON을 찾음
        start = model_text.find("{")
        end = model_text.rfind("}")
        if start != -1 and end != -1 and end > start:
            chunk = model_text[start:end+1]
        else:
            raise ValueError("모델 응답에서 JSON 블록을 찾지 못했습니다:\n" + model_text)
    
    try:
        # JSON 문자열을 파이썬 딕셔너리로 변환
        return json.loads(chunk)
    except json.JSONDecodeError:
        # 마지막 항목 뒤에 쉼표가 붙는 등 흔한 JSON 오류를 보정하여 재시도
        chunk = re.sub(r",\s*]", "]", chunk)
        chunk = re.sub(r",\s*}", "}", chunk)
        try:
            return json.loads(chunk)
        except Exception as e:
            raise ValueError(f"JSON 파싱에 실패했습니다 (오류: {e}):\n{chunk}")

### 4-3. 결과 정제 및 표준화 (Normalization)

LLM이 생성한 JSON을 MFDS 표준에 맞게 다듬고, 'evidence'(증거) 항목을 더 정확하게 보강합니다.

In [7]:
def normalize_and_fix_evidence(model_json: Dict, ocr_text: str) -> Dict:
    """
    LLM이 생성한 JSON을 MFDS 표준에 맞게 정규화하고,
    OCR 원문을 기반으로 증거(evidence)를 보강 및 수정합니다.
    """
    # 1. 표준 명칭으로 정규화 (예: '계란', '난류' -> '계란(난류)')
    normalized_allergens = []
    seen = set()
    for item in model_json.get("allergens_found", []):
        c = (item.get("canonical") or "").strip()
        if c in ["계란", "난류"]: c = "계란(난류)"
        if "조개" in c: c = "조개류"
        
        if c in MFDS_CANON:
            # 중복 제거
            key = (c, tuple(sorted(item.get("evidence", []))))
            if key not in seen:
                normalized_allergens.append(item)
                seen.add(key)
    
    # 2. OCR 원문에서 더 정확한 증거(evidence) 찾기
    
    # '...함유' 라인에서 알레르겐 단어들 추출
    contains_items = re.findall(r'([가-힣]+)\s*함유', ocr_text)
    
    # 최종 결과물 구조 초기화
    final_result = {
        "allergens_found": [],
        "uncertain_mentions": model_json.get("uncertain_mentions", []),
        "notes": model_json.get("notes", "")
    }
    
    for item in normalized_allergens:
        canonical_name = item['canonical']
        evidence = set(item.get('evidence', []))
        
        # '...함유' 라인에 표준명이 포함되어 있으면 증거로 추가
        if canonical_name in contains_items:
            evidence.add(canonical_name)
        
        # 원재료명 전체에서 표준명과 관련된 단어(동의어)들을 찾아 증거로 추가
        synonyms = {"우유": ["치즈", "버터", "크림", "유청"], "밀": ["소맥"], "대두": ["간장", "된장", "레시틴"]}
        for syn in synonyms.get(canonical_name, []):
            if syn in ocr_text:
                evidence.add(syn)

        if evidence:
            final_result["allergens_found"].append({
                "canonical": canonical_name,
                "evidence": sorted(list(evidence), key=len)
            })
    
    return final_result

### 4-4. 알레르겐 분석 파이프라인 실행 함수

위에서 정의한 함수들을 순서대로 호출하여 전체 파이프라인을 실행합니다.

In [8]:
def run_allergen_pipeline(image_path: str = None, ocr_text: str = None, save_path: str = "allergen_result.json") -> Dict:
    """알레르겐 분석 파이프라인 전체를 실행합니다."""
    # OCR 텍스트가 입력되지 않았다면, 이미지 경로로 OCR 실행
    if not ocr_text:
        if not image_path:
            raise ValueError("image_path 또는 ocr_text 중 하나는 반드시 제공해야 합니다.")
        ocr_text = detect_text_by_gcp(image_path)
        if not ocr_text:
            print("OCR 결과, 텍스트를 찾을 수 없어 분석을 종료합니다.")
            return {}

    # LLM 추론 실행
    print("LLM을 통해 알레르겐 정보를 추론합니다...")
    raw_json = llm_infer_to_json(ocr_text)

    # 결과 정제 및 표준화
    print("결과를 표준화하고 증거를 정리합니다...")
    final_result = normalize_and_fix_evidence(raw_json, ocr_text)

    # 최종 결과를 JSON 파일로 저장
    # save_path의 디렉터리가 없을 경우 생성
    output_dir = os.path.dirname(save_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(final_result, f, ensure_ascii=False, indent=2)
    print(f"✅ 알레르겐 분석 완료! 결과를 '{save_path}'에 저장했습니다.")

    return final_result

## 5. 파이프라인 2: 단순 텍스트 추출 (OCR → TXT)

이미지에서 텍스트를 추출하여 `.txt` 파일로 저장하는 간단한 파이프라인입니다.

In [9]:
def run_text_extraction_pipeline(image_path: str, save_path: str = None):
    """단순 텍스트 추출 파이프라인을 실행합니다."""
    # 1. OCR 실행
    extracted_text = detect_text_by_gcp(image_path)

    # 2. 추출된 텍스트가 있을 경우에만 파일 저장
    if extracted_text:
        output_path = save_path
        
        # 3. 저장 경로가 지정되지 않으면 이미지 파일명으로 자동 생성
        if output_path is None:
            # 예: 'sample.jpg' -> 'sample.txt'
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            output_path = f"{base_name}.txt"
        
        # 4. UTF-8 인코딩으로 파일 저장 (한글 깨짐 방지)
        output_dir = os.path.dirname(output_path)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(extracted_text)
        print(f"✅ 텍스트 추출 완료! 결과를 '{output_path}'에 저장했습니다.")

## 6. 노트북에서 직접 실행하기

이제 위에서 정의한 함수들을 직접 호출하여 파이프라인을 실행할 수 있습니다.  
아래 예시 코드의 파일 경로를 실제 분석하고 싶은 이미지 파일 경로로 변경한 후 실행하세요.

In [11]:
# --- 실행 예시 ---

# 분석할 이미지 파일 경로 (🚨 실제 파일 경로로 수정하세요)
my_image_file = "차지예_009.jpg"

# --- 예시 1: 알레르겐 분석 실행 ---
if os.path.exists(my_image_file):
    print("--- 알레르겐 분석 파이프라인 실행 ---")
    
    allergen_result = run_allergen_pipeline(
        image_path=my_image_file, 
        save_path="my_allergen_result.json"
    )
    print("\n[최종 분석 결과]")
    print(json.dumps(allergen_result, ensure_ascii=False, indent=2))
    print("-" * 35)
else:
    print(f"'{my_image_file}' 경로에 이미지 파일이 없습니다. my_image_file 변수를 수정해주세요.")


# --- 예시 2: 단순 텍스트 추출 실행 ---
if os.path.exists(my_image_file):
    print("\n--- 단순 텍스트 추출 파이프라인 실행 ---")
    run_text_extraction_pipeline(
        image_path=my_image_file, 
        save_path="my_ocr_text.txt"
    )
    print("-" * 35)
else:
    print(f"'{my_image_file}' 경로에 이미지 파일이 없습니다. my_image_file 변수를 수정해주세요.")

--- 알레르겐 분석 파이프라인 실행 ---
'차지예_009.jpg' 파일에서 텍스트를 추출합니다...
텍스트 추출 완료.
LLM을 통해 알레르겐 정보를 추론합니다...
'Qwen/Qwen2.5-7B-Instruct' 모델을 로딩합니다. 잠시 기다려주세요...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


모델 로딩 완료.
결과를 표준화하고 증거를 정리합니다...
✅ 알레르겐 분석 완료! 결과를 'my_allergen_result.json'에 저장했습니다.

[최종 분석 결과]
{
  "allergens_found": [
    {
      "canonical": "대두",
      "evidence": [
        "간장",
        "대두",
        "대두:외국산"
      ]
    },
    {
      "canonical": "밀",
      "evidence": [
        "밀"
      ]
    }
  ],
  "uncertain_mentions": [],
  "notes": "원재료명에서 대두와 밀이 함유됨을 확인."
}
-----------------------------------

--- 단순 텍스트 추출 파이프라인 실행 ---
'차지예_009.jpg' 파일에서 텍스트를 추출합니다...
텍스트 추출 완료.
✅ 텍스트 추출 완료! 결과를 'my_ocr_text.txt'에 저장했습니다.
-----------------------------------


In [12]:
# --- 실행 예시 ---

# 분석할 이미지 파일 경로
my_image_file = "김광무_147.jpg"

# --- 예시 1: 알레르겐 분석 실행 ---
if os.path.exists(my_image_file):
    print("--- 알레르겐 분석 파이프라인 실행 ---")
    
    allergen_result = run_allergen_pipeline(
        image_path=my_image_file, 
        save_path="my_allergen_result.json"
    )
    print("\n[최종 분석 결과]")
    print(json.dumps(allergen_result, ensure_ascii=False, indent=2))
    print("-" * 35)
else:
    print(f"'{my_image_file}' 경로에 이미지 파일이 없습니다. my_image_file 변수를 수정해주세요.")


# --- 예시 2: 단순 텍스트 추출 실행 ---
if os.path.exists(my_image_file):
    print("\n--- 단순 텍스트 추출 파이프라인 실행 ---")
    run_text_extraction_pipeline(
        image_path=my_image_file, 
        save_path="my_ocr_text.txt"
    )
    print("-" * 35)
else:
    print(f"'{my_image_file}' 경로에 이미지 파일이 없습니다. my_image_file 변수를 수정해주세요.")

--- 알레르겐 분석 파이프라인 실행 ---
'김광무_147.jpg' 파일에서 텍스트를 추출합니다...
텍스트 추출 완료.
LLM을 통해 알레르겐 정보를 추론합니다...
결과를 표준화하고 증거를 정리합니다...
✅ 알레르겐 분석 완료! 결과를 'my_allergen_result.json'에 저장했습니다.

[최종 분석 결과]
{
  "allergens_found": [
    {
      "canonical": "밀",
      "evidence": [
        "밀",
        "밀 함유"
      ]
    },
    {
      "canonical": "대두",
      "evidence": [
        "대두, 밀, 우유, 닭고기, 쇠고기, 조개류(굴) 함유"
      ]
    },
    {
      "canonical": "우유",
      "evidence": [
        "크림",
        "버터",
        "치즈",
        "대두, 밀, 우유, 닭고기, 쇠고기, 조개류(굴) 함유"
      ]
    },
    {
      "canonical": "닭고기",
      "evidence": [
        "대두, 밀, 우유, 닭고기, 쇠고기, 조개류(굴) 함유"
      ]
    }
  ],
  "uncertain_mentions": [],
  "notes": "원재료명에서 알레르기 유발물질을 직접 확인하였습니다."
}
-----------------------------------

--- 단순 텍스트 추출 파이프라인 실행 ---
'김광무_147.jpg' 파일에서 텍스트를 추출합니다...
텍스트 추출 완료.
✅ 텍스트 추출 완료! 결과를 'my_ocr_text.txt'에 저장했습니다.
-----------------------------------
